In [1]:
import torch
from torchvision import transforms
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np


/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowExxb
  Referenced from: <AE5A0901-5B6C-3028-ADEE-0C068D0474D9> /opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <73F684F2-E1D9-32A8-B2A3-4E00185FE48E> /opt/anaconda3/lib/python3.9/site-packages/torch/lib/libc10.dylib
  warn(f"Failed to load image Python extension: {e}")


In [6]:
import numpy
numpy.__version__

'1.26.2'